!pip install BeautifulSoup4, lxml, mapbox, pandas, requests, IPython, googlemaps


In [ ]:
import json, config
from urllib.request import urlopen
from bs4 import BeautifulSoup
import googlemaps

In [ ]:
from datetime import datetime

gmaps = googlemaps.Client(key=config.google_api_key)

from mapbox import Geocoder
geocoder = Geocoder(access_token=config.api_key)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_current_Major_League_Baseball_stadiums"

In [ ]:
page = urlopen(url)

In [ ]:
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify())

In [ ]:
first_table = soup.find(
    'table', class_='wikitable sortable plainrowheaders')


In [ ]:
A=[] #IMG
B=[] #Arena
C=[] #Capacity
D=[] #City
E=[] #Surface
F=[] #Team
G=[] #Opened
H=[] #Design
I=[] #Roof Type
J=[] #State

In [ ]:
for row in first_table.findAll('tr'):
    cells=row.findAll(['td','th'])
    if cells[0].find(scope_=lambda x: x != 'col'):
        A.append(cells[0].find('img')['src'].replace("120px", "250px"))
        B.append(cells[1].find(text=True))
        ###this splits the city and state then puts them in their own arrays.
        location = cells[3].find(text=True).split(',')
        D.append(location[0])
        J.append(location[1].lstrip())
        C.append(cells[2].find(text=True).rstrip('\n'))
        E.append(cells[4].find(text=True).rstrip('\n'))
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True).rstrip('\n'))
        H.append(cells[8].find(text=True))
        I.append(cells[9].find(text=True).rstrip('\n'))


In [ ]:
import pandas as pd
df=pd.DataFrame(F,columns=['Team'])
df['Arena']=B
df['City']=D
df['State'] = J
df['Capacity']=C
df['opened']=G
df['surface']=E
df['Roof Type']=I
df['Design']=H
df['image']=A
df

In [26]:
location = [];
for i in range(len(B)):
    searchTerm = B[i] + ", " + J[i]
    geocode_result = gmaps.geocode(searchTerm)
    data = geocode_result[0]['geometry']['location']
    tempArray = []
    tempArray.append(float(data['lng']))
    tempArray.append(float(data['lat']))
    location.append(tempArray[::-1])

[[43.0279776, -87.9711504], [33.800308, -117.8827321], [38.6226188, -90.1928209], [33.4453344, -112.0667091], [40.7570877, -73.8458213], [39.9060572, -75.1664952], [42.33899840000001, -83.0485197], [39.7558823, -104.9941781], [34.073851, -118.2399583], [42.3466764, -71.0972178], [32.7474557, -97.0838351], [39.0973808, -84.5070714], [41.8299021, -87.63375219999999], [39.0516719, -94.4803142], [25.7781487, -80.2195998], [29.7572694, -95.3555143], [38.8730102, -77.0074329], [37.7515946, -122.2005458], [37.7785951, -122.3892698], [39.2839354, -76.6215654], [32.7076035, -117.157036], [40.446941, -80.0057054], [41.496211, -81.6852289], [43.64177979999999, -79.38914369999999], [47.5914026, -122.3325085], [44.98174299999999, -93.27762919999999], [27.7682246, -82.65339209999999], [33.8910646, -84.4684498], [41.9484384, -87.6553327], [40.8296426, -73.9261745]]


response = geocoder.forward(B[0] + ", " + D[0] + ", " + J[0])
collection = response.json()
data = collection['features'][0]['geometry']['coordinates']


In [27]:
final_data = []
for i in range(len(A)):
    temp={}
    temp['Image'] = "HTTPS:" + A[i]
    temp['Team'] = F[i]
    temp['Arena'] = B[i]
    temp['Location'] = D[i] + ", " + J[i]
    temp['Capacity'] = C[i]
    temp['opened'] = G[i]
    temp['Surface'] = E[i]
    temp['Roof Type'] = I[i]
    temp['Design'] = H[i]
    temp['Coordinates'] = location[i]
    final_data.append(temp)

In [ ]:
print(len(final_data))

In [28]:
with open('MLB.json', 'w') as outfile:
    json.dump(final_data, outfile)

In [ ]:
import requests, IPython.display as Disp
url = final_data[6]['Image']
Disp.Image(requests.get(url).content)

In [ ]:
print(url)